# CARGA DEL DATASET Y LIBRERIAS

## Importacion de Librerias

In [1]:
import pandas as pd
import numpy as np
import os


#Seteos del notebook
#pd.set_option('display.max_colwidth', 40)
#pd.options.display.min_rows
#pd.set_option('display.width', 1000)

## Carga del dataset crudo para su tratamiento

In [2]:
data = []
# Leer el archivo línea por línea y cargar los objetos JSON
#with open('../raw_data/australian_users_items.json', encoding='utf-8') as json_file:
with open(f'D:\PEDRO\AHORA\A LA CARGA!!\WORKING AREA\Cursos and Tuts\Henry\Labs\PIML3\DESARROLLO DEL PI\A Local/raw_data/australian_users_items.json', encoding='utf-8') as json_file:
    for line in json_file:
        data.append(eval(line))  # Convertir la línea en un objeto Python

# Crear el DataFrame a partir de la lista de objetos JSON
df_user_items = pd.DataFrame(data)

# LIMPIEZA Y TRANSFORMACION DEL DATASET CRUDO

## Exploracion Basica

In [3]:
print('columnas en df_user_itemss: ','[' + ', '.join(df_user_items.keys()) + ']')

columnas en df_user_itemss:  [user_id, items_count, steam_id, user_url, items]


In [4]:
print('Numero de filas y campos: ',df_user_items.shape)
print('Tipos de las columnas:')
print(df_user_items.dtypes)
print('\n')

print('Datos faltantes:')
print(df_user_items.isnull().sum())
df_user_items.head(3)

Numero de filas y campos:  (88310, 5)
Tipos de las columnas:
user_id        object
items_count     int64
steam_id       object
user_url       object
items          object
dtype: object


Datos faltantes:
user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


Al parecer el campo 'items' esta anidado; checkamos su primer valor

In [5]:
print("Primer contenido de las columna 'items':")
print(df_user_items['items'][0])

Primer contenido de las columna 'items':
[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_foreve

Vemos que debemos desanidar 'items', pero primero existen tratamientos previos que realizar

## Tratamientos previos al desanidado del campo 'items'

Antes del desanidado verificamos si existen ides duplicados en 'user_id'. Para eliminar estos duplicados primero debemos verificar registros con valores faltantes en 'items', pues de nada sirven registros con ides unicos en 'user_id' y valores vacios en 'items'

### Exploramos el campo 'user_id'

Los ides deben ser unicos, por lo que verificamos si existen valores repetidos

In [6]:
#Muestra los valores unicos en 'user_id'
ides = df_user_items['user_id'] #muestra los valores en 'user_id'
print("Total de valores en 'user_id': ", len(ides))

ides_unicos = df_user_items['user_id'].unique() #muestra valores únicos en  'user_id'
print("Número de valores únicos en 'user_id': ", len(ides_unicos))

Total de valores en 'user_id':  88310
Número de valores únicos en 'user_id':  87626


Encontramos muchos valores repetidos en 'user_id'      
Para analizar estas ocurrencias vamos a capturar 3 ides que se repiten y mostrar todas las filas implicadas  
Para acelerar la tarea trabajamos sin el campo 'items'

In [7]:
# Filtrar las columnas a mantener
columnas_a_mantener = ['user_id','steam_id']
df_user_items_no_items = df_user_items[columnas_a_mantener]

# Identifica todos los user_id duplicados y cuenta sus ocurrencias
duplicados = df_user_items_no_items['user_id'].value_counts()

# Captura en una lista tres user_id con ocurrencias repetidas .
user_id_repetidos = duplicados[duplicados > 1].head(3).index.tolist()
print('Los 3 primeros ides repetidos son:',user_id_repetidos)

# Filtra y muestra las filas correspondientes a estos user_id
filas_repetidas = df_user_items_no_items[df_user_items_no_items['user_id'].isin(user_id_repetidos)]
filas_repetidas

Los 3 primeros ides repetidos son: ['X03-Suits', '76561198027488037', '76561198100326818']


,user_id,steam_id
11160,76561198100326818,76561198100326818
11161,76561198100326818,76561198100326818
14828,76561198027488037,76561198027488037
14832,X03-Suits,76561198085989695
14833,76561198027488037,76561198027488037
14837,X03-Suits,76561198085989695
29500,76561198100326818,76561198100326818
36237,76561198027488037,76561198027488037
36239,X03-Suits,76561198085989695


### Tratamiento del Campo 'items'

Antes de lidiar con los ides repetidos checkamos si algo extraño en el campo 'items' de esos 3 ides repetidos  
Para esto usamos 'df_user_items'

In [8]:
for id_repetido in user_id_repetidos:
    filas_repetida = df_user_items[df_user_items['user_id'] == id_repetido]
    print(filas_repetida)

         user_id  items_count           steam_id   
14832  X03-Suits          141  76561198085989695  \
14837  X03-Suits          141  76561198085989695   
36239  X03-Suits          141  76561198085989695   

                                     user_url   
14832  http://steamcommunity.com/id/X03-Suits  \
14837  http://steamcommunity.com/id/X03-Suits   
36239  http://steamcommunity.com/id/X03-Suits   

                                                   items  
14832  [{'item_id': '10', 'item_name': 'Counter-Strik...  
14837  [{'item_id': '10', 'item_name': 'Counter-Strik...  
36239  [{'item_id': '10', 'item_name': 'Counter-Strik...  
                 user_id  items_count           steam_id   
14828  76561198027488037           53  76561198027488037  \
14833  76561198027488037           53  76561198027488037   
36237  76561198027488037           53  76561198027488037   

                                                user_url   
14828  http://steamcommunity.com/profiles/76561198027... 

Vemos que en 'df_user_items' hay algunos datos vacios en el campo 'items'. Los eliminamos

In [9]:
print('Numero de filas y campos antes de la eliminacion: ',df_user_items.shape)
df_user_items = df_user_items[df_user_items['items'].apply(lambda x: len(x) > 0)]
print('Numero de filas y campos despues de la eliminacion: ',df_user_items.shape)


Numero de filas y campos antes de la eliminacion:  (88310, 5)
Numero de filas y campos despues de la eliminacion:  (71504, 5)


### Tratamiento del Campo 'user_id'

Ahora tratamos las filas con ides repetidos. Como estos deben ser unicos, dejamos solo un ejemplar y eliminamos el resto

In [10]:
print('Numero de filas y campos antes de la eliminacion: ',df_user_items.shape)
df_user_items = df_user_items.drop_duplicates(subset='user_id', keep='first')
print('Numero de filas y campos despues de la eliminacion: ',df_user_items.shape)

Numero de filas y campos antes de la eliminacion:  (71504, 5)
Numero de filas y campos despues de la eliminacion:  (70912, 5)


Verificaciones las veces que se repiten los valores de 'user_id'


In [11]:
repeticiones = df_user_items['user_id'].value_counts()
print("Conteo de valores repetidos en 'user_id':")
print(repeticiones)

Conteo de valores repetidos en 'user_id':
user_id
76561197970982479    1
76561198054378529    1
76561198054394508    1
dropbearloc          1
76561198054389225    1
                    ..
76561198036292321    1
76561198047246195    1
boynextdoor7354      1
pooinmypants         1
76561198329548331    1
Name: count, Length: 70912, dtype: int64


## Desanidando el campo 'items'

In [12]:
print('columnas en df_user_items: ','[' + ', '.join(df_user_items.keys()) + ']')

columnas en df_user_items:  [user_id, items_count, steam_id, user_url, items]


In [13]:
print('Numero de filas y campos antes del desanidado: ',df_user_items.shape)
print('columnas antes del desanidado: ','[' + ', '.join(df_user_items.keys()) + ']')
print('Datos faltantes antes del desanidado:')
print(df_user_items.isnull().sum())
print('\n')

data_desanidada = []

for index, row in df_user_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items_list = row['items']

    for item in items_list:
        new_row = {
            'user_id': user_id,
            'items_count': items_count,
            'steam_id': steam_id,
            'user_url': user_url,
            'item_id': item.get('item_id', ''),
            'item_name': item.get('item_name', ''),
            'playtime_forever': item.get('playtime_forever', ''),
            'playtime_2weeks': item.get('playtime_2weeks', '')
        }

        data_desanidada.append(new_row)

df_user_items = pd.DataFrame(data_desanidada)

print('Numero de filas y campos despues del desanidado: ',df_user_items.shape)
print('columnas despues del desanidado: ','[' + ', '.join(df_user_items.keys()) + ']')
print('Datos faltantes despues del desanidado:')
print(df_user_items.isnull().sum())

Numero de filas y campos antes del desanidado:  (70912, 5)
columnas antes del desanidado:  [user_id, items_count, steam_id, user_url, items]
Datos faltantes antes del desanidado:
user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64




Numero de filas y campos despues del desanidado:  (5094082, 8)
columnas despues del desanidado:  [user_id, items_count, steam_id, user_url, item_id, item_name, playtime_forever, playtime_2weeks]
Datos faltantes despues del desanidado:
user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64


## Tratamiento del campo 'item_id'

El campo 'item_id' es el campo mas importante del dataset despues del campo 'user_id'.  
Cada usuario debe haber reseñado sus items una sola vez, por lo que nos aseguramos que para cada 'user_id' no haya valores repetidos del campo 'item_id'.

### Exploramos 'item_id'

In [14]:
print('columnas en df_user_items: ','[' + ', '.join(df_user_items.keys()) + ']')


columnas en df_user_items:  [user_id, items_count, steam_id, user_url, item_id, item_name, playtime_forever, playtime_2weeks]


In [15]:
print(df_user_items.shape)
print("Tipo de dato en 'item_id':",df_user_items['item_id'].dtype)
df_user_items.head(3)

(5094082, 8)
Tipo de dato en 'item_id': object


,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0


Todos los valores en la columna 'item_id' deben ser digitos, es decir con formato tipo `r'^\d+$'`  
Verificamos si hay valores que no cumplan tal formato

In [16]:
# Busca valores en 'item_id' que no sigan el formato deseado
valores_no_validos = df_user_items[~df_user_items['item_id'].astype(str).str.contains(r'^\d+$')]

print("Valores en 'item_id' sin el formato 'r'^\d+$': ", len(valores_no_validos['item_id'])) #muestra la cantidad de valores sin el formato

Valores en 'item_id' sin el formato 'r'^\d+$':  0


Checkamos si hay valores repetidos en 'item_id'. Para eso capturamos tres ides repetidos en 'user_id'

In [17]:
# Filtrar las columnas a mantener
columnas_a_mantener = ['user_id','item_id']
df_user_items_no_full = df_user_items[columnas_a_mantener]# Identifica todos los user_id duplicados y cuenta sus ocurrencias
duplicados = df_user_items_no_full['user_id'].value_counts()

# Captura en una lista tres user_id con ocurrencias repetidas .
user_id_repetidos = duplicados[duplicados > 1].head(3).index.tolist()

# Filtra y muestra las filas correspondientes a estos user_id
filas_repetidas = df_user_items_no_full[df_user_items_no_full['user_id'].isin(user_id_repetidos)]

for id_repetido in user_id_repetidos:
    filas_repetida = df_user_items_no_full[df_user_items['user_id'] == id_repetido]
    print(filas_repetida)

        user_id item_id
345620  phrostb     300
345621  phrostb     240
345622  phrostb      10
345623  phrostb      30
345624  phrostb      40
...         ...     ...
353377  phrostb  526790
353378  phrostb  527510
353379  phrostb  527520
353380  phrostb  527810
353381  phrostb  324810

[7762 rows x 2 columns]
              user_id item_id
1893067  thugnificent      10
1893068  thugnificent      20
1893069  thugnificent      30
1893070  thugnificent      40
1893071  thugnificent      50
...               ...     ...
1899762  thugnificent  524340
1899763  thugnificent  527510
1899764  thugnificent  527520
1899765  thugnificent  527810
1899766  thugnificent  528660

[6700 rows x 2 columns]
       user_id item_id
98435   chidvd      10
98436   chidvd      80
98437   chidvd     100
98438   chidvd      30
98439   chidvd     320
...        ...     ...
104840  chidvd  524030
104841  chidvd  394140
104842  chidvd  527440
104843  chidvd  324810
104844  chidvd  529670

[6410 rows x 2 columns]


A simple vista no se ven valores repetidos en 'item_id'. Nos aseguramos  
Verificamos si existen 'item_id' repetidos en las filas que tienen el mismo 'user_id'

In [18]:
duplicados = df_user_items[df_user_items.duplicated(subset=['user_id', 'item_id'], keep=False)]
print('Cantidad de "item_id" repetidos por cada usuario: ',len(duplicados))

Cantidad de "item_id" repetidos por cada usuario:  0


## Tratamiento del campo 'playtime_forever'

### Exploramos 'playtime_forever'

In [19]:
print(df_user_items.shape)
print("Tipo de dato en 'playtime_forever':",df_user_items['playtime_forever'].dtype)
print('columnas en df_user_items: ','[' + ', '.join(df_user_items.keys()) + ']')
print("Cantidad de valores faltantes en 'playtime_forever':",df_user_items['playtime_forever'].isnull().sum())

(5094082, 8)
Tipo de dato en 'playtime_forever': int64
columnas en df_user_items:  [user_id, items_count, steam_id, user_url, item_id, item_name, playtime_forever, playtime_2weeks]
Cantidad de valores faltantes en 'playtime_forever': 0


Mostramos los valores unicos de 'price'

In [20]:
unique_prices = df_user_items['playtime_forever'].unique()
print("Número de valores únicos en 'playtime_forever': ", len(unique_prices))

print("Número de valores únicos en 'playtime_forever': ", len(unique_prices))

# Seleccionar aleatoriamente 80 valores únicos de la serie
selected_values = np.random.choice(unique_prices, 200, replace=False)

# Dar forma a la matriz de 8x10
matriz = selected_values.reshape(20, 10)

# Imprimir la matriz
print(matriz)


Número de valores únicos en 'playtime_forever':  48861
Número de valores únicos en 'playtime_forever':  48861
[[  8209  19848  40768 142052   1559  55766  75746  89284  66528  16501]
 [ 29553  36939  42188  19178   1299  21177 138407  12488  38293  40507]
 [   846  51656 110734   8108  41266  11413  13433  48018  39050  12540]
 [ 42637  14522  34839     65  18523  48014  14949  39620   4404  61144]
 [ 12937  10211  65201  17034    233  72725  65113   9347  22207  27416]
 [ 41231  51458   2969  28576 125805  36374  42888 145626  21265  37248]
 [  1330  32741  22206  19997  94728  39173  22414  30540  20362  54049]
 [ 29502  62129  28018  66703  18415  17174   7483  63490  86716  39717]
 [  6492  29875  33219  77648  31107  31846   8556  81788  15777  20426]
 [  6190   2772  37657  38051  91503   1336  63394  18941  53168   4201]
 [ 12447  36064  36582  23527   5735  21752   4968   2115  40945  94731]
 [  4762  65044  79785  14874  37543 152610  63754  38514 103171   6147]
 [  7747  2506

Al parecer los valores son validos, nos asegurmos.  
Como todos los valores deben ser digitos, es decir con formato tipo `r'^\d+$'`, verificamos si hay valores que no cumplan tal formato

In [21]:
# Busca valores en 'item_id' que no sigan el formato deseado
valores_no_validos = df_user_items[~df_user_items['playtime_forever'].astype(str).str.contains(r'^\d+$')]

print("Valores en 'playtime_forever' sin el formato 'r'^\d+$': ", len(valores_no_validos['playtime_forever'])) #muestra la cantidad de valores sin el formato

Valores en 'playtime_forever' sin el formato 'r'^\d+$':  0


## Guardamos el dataset preparado

In [22]:
#df_user_items.to_csv('../Data/df_user_items_prepared.csv', index=False)

# CREACION DE LOS ENPOINT DATASETS

Terminada la preparacion de los datos en 'df_user_items' toca crear los diferentes datasets en formato parquet que van alimentar al servidor de consultas

## Endpoint `'def userdata(user_id:str)'`

### Transformacion

Filtramos los campos 'user_id' y 'item_id'. Eliminamos el resto

In [77]:
columnas_a_mantener = ['user_id', 'item_id']
df_user_items_userdata_user_id_respuesta1 = df_user_items[columnas_a_mantener]

print(df_user_items_userdata_user_id_respuesta1.shape)
print(df_user_items_userdata_user_id_respuesta1.dtypes)
df_user_items_userdata_user_id_respuesta1.head(3)

(5094082, 2)
user_id    object
item_id    object
dtype: object


,user_id,item_id
0,76561197970982479,10
1,76561197970982479,20
2,76561197970982479,30


Aligeramos el dataframe para evitar problemas de memoria en el deployment

In [78]:
print('Numero de filas y campos antes de la eliminacion: ',df_user_items_userdata_user_id_respuesta1.shape)
valores_permitidos = ['76561198092473445', 'junaid_m_sam']

# Crea una máscara booleana para identificar los registros que cumplen con la condición
mascara = df_user_items_userdata_user_id_respuesta1['user_id'].isin(valores_permitidos)

# Usa la máscara para filtrar el DataFrame y mantener solo los registros deseados
df_user_items_userdata_user_id_respuesta1 = df_user_items_userdata_user_id_respuesta1[mascara]

print('Numero de filas y campos despues de la eliminacion: ',df_user_items_userdata_user_id_respuesta1.shape)

Numero de filas y campos antes de la eliminacion:  (5094082, 2)
Numero de filas y campos despues de la eliminacion:  (420, 2)


### To Parquet

In [75]:
df_user_items_userdata_user_id_respuesta1.to_parquet('../Data/df_user_items_userdata_user_id_respuesta1.parquet')

## Endpoint `'def UserForGenre(genero: str)'`

### Transformacion

Filtramos los campos 'user_id' y 'item_id'. Eliminamos el resto

In [69]:
columnas_a_mantener = ['user_id', 'item_id', 'playtime_forever']
df_user_items_userforgenre_genero = df_user_items[columnas_a_mantener]

print(df_user_items_userforgenre_genero.shape)
print(df_user_items_userforgenre_genero.dtypes)
df_user_items_userforgenre_genero.head(3)

(5094082, 3)
user_id             object
item_id             object
playtime_forever     int64
dtype: object


,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7


Aligeramos el dataframe para evitar problemas de memoria en el deployment

In [70]:
print('Numero de filas y campos antes de la eliminacion: ',df_user_items_userforgenre_genero.shape)

df_user_items_userforgenre_genero = df_user_items_userforgenre_genero[(df_user_items_userforgenre_genero['playtime_forever'] < 0) | (df_user_items_userforgenre_genero['playtime_forever'] > 100)]

conteo_user_id = df_user_items_userforgenre_genero['user_id'].value_counts()

# Crea una máscara booleana para identificar valores que se repiten de 1 a 10 veces
mascara = (conteo_user_id[df_user_items_userforgenre_genero['user_id']] >= 1) & (conteo_user_id[df_user_items_userforgenre_genero['user_id']] <= 10)

mascara.index = df_user_items_userforgenre_genero.index

# Usa la máscara para filtrar el DataFrame y eliminar las filas correspondientes
df_user_items_userforgenre_genero = df_user_items_userforgenre_genero[~mascara]


conteo_item_id = df_user_items_userforgenre_genero['item_id'].value_counts()

# Crea una máscara booleana para identificar valores que se repiten de 1 a 10 veces
mascara = (conteo_item_id[df_user_items_userforgenre_genero['item_id']] >= 1) & (conteo_item_id[df_user_items_userforgenre_genero['item_id']] <= 10)

mascara.index = df_user_items_userforgenre_genero.index

# Usa la máscara para filtrar el DataFrame y eliminar las filas correspondientes
df_user_items_userforgenre_genero = df_user_items_userforgenre_genero[~mascara]

print('Numero de filas y campos despues de la eliminacion: ',df_user_items_userforgenre_genero.shape)


Numero de filas y campos antes de la eliminacion:  (5094082, 3)
Numero de filas y campos despues de la eliminacion:  (1917974, 3)


### To Parquet

In [71]:
df_user_items_userforgenre_genero.to_parquet('../Data/df_user_items_userforgenre_genero.parquet', index=False)

## Endpoint `'def recomendacion_usuario(user_id: srt)'`

A continuacion preparamos el dataset que alimentará a la etapa de preprocesamiento de datos del sistema de recomendacion

### Transformaciones

Filtramos los campos 'user_id', 'item_id' y 'item_name'. Eliminamos el resto

In [ ]:
columnas_a_mantener = ['user_id','item_id','item_name']
df_user_items_recommendacion_usuario_user_id = df_user_items[columnas_a_mantener]

print(df_user_items_recommendacion_usuario_user_id.shape)
print(df_user_items_recommendacion_usuario_user_id.dtypes)
print('Datos faltantes:')
print(df_user_items_recommendacion_usuario_user_id.isnull().sum())
df_user_items_recommendacion_usuario_user_id.head(3)

(5094082, 3)
user_id      object
item_id      object
item_name    object
dtype: object
Datos faltantes:
user_id      0
item_id      0
item_name    0
dtype: int64


,user_id,item_id,item_name
0,76561197970982479,10,Counter-Strike
1,76561197970982479,20,Team Fortress Classic
2,76561197970982479,30,Day of Defeat


Nos cercioramos que en 'user_id', 'item_id' y 'item_name' no haya ningun registro con campos vacios 

In [ ]:
print('Numero de filas antes de la eliminacion: ',df_user_items_recommendacion_usuario_user_id.shape)

df_user_items_recommendacion_usuario_user_id = df_user_items_recommendacion_usuario_user_id[
    (df_user_items_recommendacion_usuario_user_id['user_id'].apply(lambda x: len(x) > 0)) &
    (df_user_items_recommendacion_usuario_user_id['item_id'].apply(lambda x: len(x) > 0)) &
    (df_user_items_recommendacion_usuario_user_id['item_name'].apply(lambda x: len(x) > 0))
]

print('Numero de filas despues de la eliminacion: ',df_user_items_recommendacion_usuario_user_id.shape)

Numero de filas antes de la eliminacion:  (5094082, 8)
Numero de filas despues de la eliminacion:  (5094082, 3)


### To Parquet

In [ ]:
df_user_items_recommendacion_usuario_user_id.to_parquet('../Data/df_user_items_recommendacion_usuario_user_id.parquet', index=False)

# Codigo no relacionado al proyecto

## Listar variables en memoria 

In [ ]:
import builtins
import sys

# Obtener la lista de nombres de variables
variable_names = dir()
variables_descartables = ['exit','quit','Out','In','sys','var','var_name','variable_names','builtins','contenido','size','user_defined_variables','variables_descartables']
# Filtrar las variables creadas manualmente
user_defined_variables = [
    var for var in variable_names 
    #if not var.startswith('_')
    if var not in dir(builtins) and not var.startswith('_') and var not in variables_descartables
]
#print(user_defined_variables)
print('Variables encontradas: ', len(user_defined_variables))
# Imprimir el tamaño y la fecha de creación de las variables
for var_name in user_defined_variables:
    var = globals()[var_name]
    if sys.getsizeof(var) >= 100000:
        size = sys.getsizeof(var)  
        contenido = eval(var_name)
        print(f"La variable {var_name} pesa {size} bytes y es de tipo {str(type(contenido))[8:-2]}")

Variables encontradas:  26
La variable df_user_items pesa 71310066 bytes y es de tipo pandas.core.frame.DataFrame
La variable df_user_items_userdata_prepared_desanidado pesa 2088152303 bytes y es de tipo pandas.core.frame.DataFrame
La variable filas_repetidas1 pesa 1438635 bytes y es de tipo pandas.core.frame.DataFrame
La variable ides_unicos pesa 567408 bytes y es de tipo numpy.ndarray
